In [1]:
!pip install -q sklearn

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
import pathlib

dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder-mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

In [4]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [5]:
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

In [6]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

7383 train examples
1846 validation examples
2308 test examples


In [7]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [8]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [9]:
for feature_batch, label_batch in train_ds.take(1):
  print("Every feature:", list(feature_batch.keys()))
  print("A batch of ages:", feature_batch["Age"])
  print("A batch of targets:", label_batch)

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']
A batch of ages: tf.Tensor([ 1  2 18 12  3], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 1 0 0 1], shape=(5,), dtype=int64)


In [10]:
example_batch = next(iter(train_ds))[0]

In [11]:
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [12]:
photo_count = feature_column.numeric_column("PhotoAmt")
demo(photo_count)

[[3.]
 [4.]
 [8.]
 [2.]
 [8.]]


In [13]:
age = feature_column.numeric_column("Age")
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 3, 5])
demo(age_buckets)

[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [14]:
animal_type = feature_column.categorical_column_with_vocabulary_list(
    'Type', ['Cat', 'Dog'])

animal_type_one_hot = feature_column.indicator_column(animal_type)
demo(animal_type_one_hot)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [15]:
breed1 = feature_column.categorical_column_with_vocabulary_list(
    'Breed1', dataframe.Breed1.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
demo(breed1_embedding)

[[-0.11592568 -0.20752071 -0.11291786 -0.29825068  0.50073797  0.04140102
   0.09096944 -0.4337509 ]
 [ 0.18161589  0.24248537 -0.53007144 -0.251281    0.00198221  0.01056604
   0.0036837   0.02454562]
 [ 0.18161589  0.24248537 -0.53007144 -0.251281    0.00198221  0.01056604
   0.0036837   0.02454562]
 [ 0.18161589  0.24248537 -0.53007144 -0.251281    0.00198221  0.01056604
   0.0036837   0.02454562]
 [ 0.18161589  0.24248537 -0.53007144 -0.251281    0.00198221  0.01056604
   0.0036837   0.02454562]]


In [17]:
breed1_hashed = feature_column.categorical_column_with_hash_bucket(
    'Breed1', hash_bucket_size=10)
demo(feature_column.indicator_column(breed1_hashed))

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [18]:
crossed_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [19]:
feature_columns = []

for header in ['PhotoAmt', 'Fee', 'Age']:
  feature_columns.append(feature_column.numeric_column(header))

In [20]:
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 2, 3, 4, 5])
feature_columns.append(age_buckets)

In [22]:
indicator_column_names = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                          'FurLength', 'Vaccinated', 'Sterilized', 'Health']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [23]:
breed1 = feature_column.categorical_column_with_vocabulary_list(
    'Breed1', dataframe.Breed1.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
feature_columns.append(breed1_embedding)

In [24]:
age_type_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=100)
feature_columns.append(feature_column.indicator_column(age_type_feature))

In [25]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [26]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [27]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(1)                             
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
231/231 [==============================] - 4s 17ms/step - loss: 0.6932 - accuracy: 0.6860 - val_loss: 0.5102 - val_accuracy: 0.7405
Epoch 2/10
231/231 [==============================] - 3s 14ms/step - loss: 0.5653 - accuracy: 0.7106 - val_loss: 0.5000 - val_accuracy: 0.7443
Epoch 3/10
231/231 [==============================] - 3s 14ms/step - loss: 0.5221 - accuracy: 0.7255 - val_loss: 0.4996 - val_accuracy: 0.7362
Epoch 4/10
231/231 [==============================] - 3s 14ms/step - loss: 0.5060 - accuracy: 0.7344 - val_loss: 0.4910 - val_accuracy: 0.7438
Epoch 5/10
231/231 [==============================] - 3s 14ms/step - loss: 0.5013 - accuracy: 0.7359 - val_loss: 0.4955 - val_accuracy: 0.7183
Epoch 6/10
231/231 [==============================] - 3s 14ms/step - loss: 0.4917 - accuracy: 0.7394 - val_loss: 0.5096 - val_accuracy: 0.7335
Epoch 7/10
231/231 [==============================] - 3s 14ms/step - loss: 0.4832 - accuracy: 0.7427 - val_loss: 0.5014 - val_accuracy: 0.7037

In [28]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

73/73 [==============================] - 1s 9ms/step - loss: 0.5216 - accuracy: 0.7101
Accuracy 0.7101386189460754
